# Simulating Sales Data

### Customer Information Data Set
- Customer ID
- First Name
- Last Name
- Age
- Gender (Female, Male, Non-Binary, Prefer Not to Say)
- Zip Code Location
- Custoemr Lifetime Value (CLV)
- Customer Segment (Through analysis: high spenders, occasional buyers, loyal customers)
- Feedback and Ratings

### Sales Transaction Data
- Transaction ID
- Transaction Date Time
- Type (Sale or Refund)
- Store ID
- Customer ID
- Purchase Total Amount
- Payment Method (Credit Card, Debit Card, Gift Card, Cash)
- Coupon/Promotion Code In Order

### Sales Transaction Details
- Transaction ID
- Product ID
- Quantity Purchased
- CouponCode

### Product Details
- Product ID
- Product Description
- Product Category (electronics, clothing, groceries)


In [101]:
# Imports

import re
import pandas as pd
import numpy as np
import uuid
import datetime
import random

---
# Customer Information

#### Gender Neutral Names
https://www.emmasdiary.co.uk/baby-names/our-top-300-unisex-baby-names
#### Last Names
https://www.rong-chang.com/namesdict/100_last_names.htm#google_vignette

In [79]:
# Open the file in read mode
with open('Data/GenderNeutralNames.csv','r')as file:
    # read the names from the file and remove new line characters
    excel_list = [line.strip() for line in file.readlines()]
    names = [row.replace('Â\xa0','').replace(" ","").split('.')[-1] for row in excel_list if any(char.isdigit() for char in row)]
#names

In [68]:
# Open the file in read mode
with open('Data/LastNames.csv','r')as file:
    # read the names from the file and remove new line characters
    excel_list = [line.strip() for line in file.readlines()]
# Process Names
last_names = []
for row in excel_list:
    # remove unwanted characters (Â\xa0, extra spaces, and consecutive commas)
    cleaned_row = row.replace('Â\xa0', '').replace(" ", "").replace(',,', '')
    
    # Extract the text after the last digit
    if any(char.isdigit() for char in cleaned_row):
        last_digit_idx = max([i for i, char in enumerate(cleaned_row) if char.isdigit()])
        extracted_name = cleaned_row[last_digit_idx+1:]
        # add the name to the last name list
        last_names.append(extracted_name)
        
print(last_names)

['Smith', 'Johnson', 'Williams', 'Jones', 'Brown', 'Davis', 'Miller', 'Wilson', 'Moore', 'Taylor', 'Anderson', 'Thomas', 'Jackson', 'White', 'Harris', 'Campbell', 'Parker', 'Evans', 'Edwards', 'Collins', 'Stewart', 'Sanchez', 'Morris', 'Rogers', 'Reed', 'Cook', 'Morgan', 'Bell', 'Murphy', 'Bailey', 'Rivera', 'Cooper', 'Richardson', 'Cox', 'Martin', 'Thompson', 'Garcia', 'Martinez', 'Robinson', 'Clark', 'Rodriguez', 'Lewis', 'Lee', 'Walker', 'Hall', 'Allen', 'Young', 'Hernandez', 'King', 'Howard', 'Ward', 'Torres', 'Peterson', 'Gray', 'Ramirez', 'James', 'Watson', 'Brooks', 'Kelly', 'Sanders', 'Price', 'Bennett', 'Wood', 'Barnes', 'Ross', 'Henderson', 'Coleman', 'Jenkins', 'Wright', 'Lopez', 'Hill', 'Scott', 'Green', 'Adams', 'Baker', 'Gonzalez', 'Nelson', 'Carter', 'Mitchell', 'Perez', 'Roberts', 'Turner', 'Phillips', 'Perry', 'Powell', 'Long', 'Patterson', 'Hughes', 'Flores', 'Washington', 'Butler', 'Simmons', 'Foster', 'Gonzales', 'Bryant', 'Alexander', 'Russell', 'Griffin', 'Diaz', 

In [69]:
zips = pd.read_csv('Data/uszips.csv')
continental_zips_df = zips[~zips['state_name'].isin(['Puerto Rico','Virgin Islands'])]
continental_zips_df['zip_6'] = continental_zips_df['zip'].apply(lambda x: '{:05}'.format(x)) #astype(str).str.zfill(5)
zip_list = list(continental_zips_df.zip_6)
#zip_list

C:\Users\Stephanie\AppData\Local\Temp\ipykernel_16796\1273500643.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  continental_zips_df['zip_6'] = continental_zips_df['zip'].apply(lambda x: '{:05}'.format(x)) #astype(str).str.zfill(5)


In [83]:
# set a random state for consistent results
np.random.seed(42)

# Assuming 1000 unique customers

# Number of records in the dataset
num_records = 1000
num_cust = 200


# Generate data for customer information
customer_data = {
    'Customer_ID': [str(i).zfill(3) for i in np.arange(1,num_cust+1)], # 200 customers
    'First_Name': np.random.choice(names,size=num_cust,replace=True),
    'Last_Name': np.random.choice(last_names,size=num_cust,replace=True),
    'Age': np.random.randint(18,85, size = num_cust),
    'Gender': np.random.choice(['Male','Female','Non-Binary','Prefer Not To Say'],
                               size = num_cust,
                               replace = True,
                               p= [0.45,0.45,0.05,0.05]),
    'Location': np.random.choice(zip_list, size=num_cust, replace=True)
}
## Variables to incorporate later ##
# Customer Lifetime Value (CLV)
# Customer Segment (Through analysis: high spenders, occasional buyers, loyal customers)
# Feedback and Ratings


# generate sales data


In [84]:
customer_data_df = pd.DataFrame(customer_data)
customer_data_df.head(3)

,Customer_ID,First_Name,Last_Name,Age,Gender,Location
0,001,Flynn,Kelly,23,Male,07718
1,002,Terry,Thompson,64,Male,39355
2,003,Gene,Edwards,72,Female,06850


# Creating Product Data

In [120]:
# customer ids
customer_ids = customer_data_df.Customer_ID

# product details
num_products = 100
product_ids = [str(i).zfill(3) for i in range(1,num_products+1)]
product_categories = ['clothing','shoes','accessories','home','beauty','personal care','office supplies','electronics','groceries']

# transaction details
num_transactions = 10000
transaction_ids = [str(uuid.uuid4()) for _ in range(num_transactions)]

# generate random dates within a specific range
start_date = datetime.datetime(2022,10,31)
end_date = datetime.datetime(2023,10,31)
transaction_dates = [start_date+datetime.timedelta(days=random.randint(1,365)) for _ in range(num_transactions)]

# generate sales_transaction_data
sale_percent = 0.9
credit_card = 0.75
debit_card = 0.15
cash = 0.05
gift_card = 0.05 
coupon_percent = 0.25
sales_transaction_data = {
    'TransactionID': transaction_ids,
    'TransactionDateTime': transaction_dates,
    'Type': [np.random.choice(['sale','refund'], p = [sale_percent, 1-sale_percent]) for _ in range(num_transactions)],
    'CustomerID': random.choices(customer_ids, k=num_transactions),
    'PurchaseTotalAmount': [round(random.uniform(1,500),2) for _ in range(num_transactions)],
    'PaymentMethod': random.choices(['credit card','debit card', 'gift card','cash'],weights=(credit_card,debit_card,gift_card,cash),k=num_transactions),
    'CouponUsed': [np.random.choice([True,False], p=(coupon_percent, 1-coupon_percent)) for _ in range(num_transactions)]    
}

# generate sales_transaction_details in which multiple products can be purchased
sales_transaction_details = []
no_coupon_percentage = 0.9
max_prod_purchased = 10
for transaction_id in transaction_ids: 
    num_products_purchased = random.randint(1, max_prod_purchased)  # Random number of products in each transaction
    products_purchased = random.sample(product_ids, num_products_purchased)
    quantities = [random.randint(1, 5) for _ in range(num_products_purchased)],
    coupon_codes = random.choices([None,'PercentOff','bogo'],weights=(no_coupon_percentage, (1-no_coupon_percentage)/2,(1-no_coupon_percentage)/2), k=num_products_purchased)
    for product, quantity,coupon_code in zip(products_purchased, quantities,coupon_codes):
        sales_transaction_details.append({
            'TransactionID': transaction_id,
            'ProductID': product,
            'QuantityPurchased': quantity,
            'CouponCodeUsed': coupon_code
        })

# Generate product details
product_details = {
    'ProductID': product_ids,
    'ProductDescription': [f'Product {i}' for i in range(1, num_products + 1)],
    'ProductCategory': random.choices(product_categories, k=num_products)
}


In [121]:
sales_transaction_df = pd.DataFrame(sales_transaction_data)
sales_transaction_details_df = pd.DataFrame(sales_transaction_details)
product_details_df = pd.DataFrame(product_details)

In [123]:
sales_transaction_df.head(3)

,TransactionID,TransactionDateTime,Type,CustomerID,PurchaseTotalAmount,PaymentMethod,CouponUsed
0,4b683b9a-9bcb-40c5-bfcd-00f5bca218f6,2023-01-05,sale,028,59.66,cash,False
1,546003b3-e7b6-4299-a20c-3bda19d54680,2023-05-30,sale,032,343.52,debit card,False
2,502dc22f-bc5c-4c13-b09f-57c0ca788b03,2022-12-19,sale,080,260.01,credit card,False


In [124]:
sales_transaction_details_df.head(3)

,TransactionID,ProductID,QuantityPurchased,CouponCodeUsed
0,4b683b9a-9bcb-40c5-bfcd-00f5bca218f6,023,"[5, 4, 3, 5, 3, 4, 4, 2, 1]",None
1,546003b3-e7b6-4299-a20c-3bda19d54680,016,[4],None
2,502dc22f-bc5c-4c13-b09f-57c0ca788b03,057,"[4, 3, 1, 5, 3]",None
